In [ ]:
from auto

In [ ]:
schema_agent = AssistantAgent(
    name="SchemaAgent",
    llm_config={"config_list": config_list},
    system_message="Your task is to inspect the database and generate a structured schema, including table names, columns, and data types."
)


sql_generation_agent = AssistantAgent(
    name="SQLGenerationAgent",
    llm_config={"config_list": config_list},
    system_message="Your task is to generate a valid SQL query based on the given schema and user query. "
                   "Ensure that table and column names are used correctly and follow standard SQL syntax."
)

sql_error_correction_agent = AssistantAgent(
    name="SQLErrorCorrectionAgent",
    llm_config={"config_list": config_list},
    system_message="Your task is to correct SQL query errors based on the given error message. "
                   "Ensure the corrected query follows proper syntax and matches the schema."
)

result_formatting_agent = AssistantAgent(
    name="ResultFormattingAgent",
    llm_config={"config_list": config_list},
    system_message="Format the SQL result in a human-readable way matching the user's query intent."
)

visualization_agent = AssistantAgent(
    name="VisualizationAgent",
    llm_config={"config_list": config_list},
    system_message="Your job is to generate Python visualization code using Matplotlib or Seaborn "
                   "based on the SQL result and the user's query."
)

dependency_checker_agent = AssistantAgent(
    name="DependencyCheckerAgent",
    llm_config={"config_list": config_list},
    system_message="Your task is to ensure all required Python packages are installed. "
                   "If any package is missing, install it dynamically when needed."
)

In [ ]:
import subprocess
import sys
import re
from autogen import AssistantAgent


def install_package(package_name):
    """
    Installs a missing package using pip.
    """
    print(f"🔹 Installing missing package: {package_name}")
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])



def execute_visualization(user_query, sql_result):
    """
    Generates visualization code using VisualizationAgent and ensures dependencies are installed dynamically.
    """
    # Step 1: Generate Python visualization code from the agent
    response = visualization_agent.generate_reply([
        {"role": "user", "content": f"User Query: {user_query}"},
        {"role": "user", "content": f"SQL Result: {sql_result}"},
        {"role": "user", "content": "Generate Python visualization code using Matplotlib or Seaborn."}
    ])
    
    # Step 2: Extract Python code from response
    match = re.search(r"python\n(.*?)\n", response, re.DOTALL)
    
    if match:
        python_code = match.group(1)
        # Step 3: Try executing the extracted Python code
        exec_globals = {}
        try:
            exec(python_code, exec_globals)
        except ImportError as e:
            missing_package = str(e).split("'")[1]  # Extract the missing package name
            print(f"\n⚠️ Missing package detected: {missing_package}")
            
            # Step 4: Ask the DependencyCheckerAgent to install the missing package
            dependency_checker_agent.generate_reply([
                {"role": "user", "content": f"Install the missing package: {missing_package}"}
            ])
            install_package(missing_package)

            # Step 5: Retry executing the visualization code
            print("\n🔄 Retrying visualization after installing the missing package...\n")
            exec(python_code, exec_globals)
    else:
        print("\n⚠️ No valid Python code found in response.")




In [ ]:
def fetch_schema(DB_PATH):
    """Fetches the schema of the database (table names, columns, and types)."""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    schema = {}
    for table in tables:
        table_name = table[0]
        cursor.execute(f"PRAGMA table_info({table_name});")  # Get column details
        columns = cursor.fetchall()
        schema[table_name] = [{"name": col[1], "type": col[2]} for col in columns]

    conn.close()

    schema_str = "\n".join(
    [
        "Table: {}\nColumns: {}".format(
            table, ", ".join(["{} ({})".format(col["name"], col["type"]) for col in cols])
        )
        for table, cols in schema.items()
    ]
    )
    return schema_str

import sqlite3

def validate_sql(db_name, sql_query):
    try:
        db_connection = sqlite3.connect(db_name)
        cursor = db_connection.cursor()
        cursor.execute(sql_query)
        result = cursor.fetchall()
        return {"success": True, "data": result}
    except Exception as e:
        return {"success": False, "error": str(e)}



def generate_sql(schema, user_query):
    response = sql_generation_agent.generate_reply(
        messages=[
            {"role": "user", "content": f"Given the schema:\n{schema}\nGenerate a SQL query for:\n{user_query}. Only output the SQL query without 
sql and last
"}
        ]
    )
    return response


def correct_sql(schema, original_query, error_message):
    response = sql_error_correction_agent.generate_reply(
        messages=[
            {"role": "user", "content": f"Schema:\n{schema}\nOriginal Query:\n{original_query}\nError:\n{error_message}\nFix the SQL query."}
        ]
    )
    return response

def format_sql_result(user_query, sql_result):
    """
    Uses the ResultFormattingAgent to refine the output in a structured user-content format.
    """
    messages = [
        {"role": "user", "content": f"User Query: {user_query}"},
        {"role": "user", "content": f"Raw SQL Result: {sql_result}"},
        {"role": "user", "content": "Format the result in a clear, human-readable way that directly answers the user's request."}
    ]
    response = result_formatting_agent.generate_reply(messages)
    return response

In [ ]:
def execution_workflow(db_path, user_query, max_retries=5):
    schema_data = fetch_schema(db_path)  # Get the schema from the database
    
    response = schema_agent.generate_reply(
        messages=[{"role": "user", "content": f"The database schema is:\n{schema_data}"}]
    )
    
    sql_query = generate_sql(response, user_query)
    
    for attempt in range(max_retries):
        validation_result = validate_sql(db_path, sql_query.strip())
        
        if validation_result["success"]:
            result = validation_result["data"]
            formatted_response = format_sql_result(user_query, result)
            return execute_visualization(user_query, formatted_response)
        else:
            sql_query = correct_sql(sql_query, user_query, validation_result["error"])
    
    raise ValueError(f"Failed to generate a valid SQL query after {max_retries} attempts. Last error: {validation_result['error']}")

formatted_result = execution_workflow("blinkit_sales.db", "What are the top 5 best-selling products based on total sales quantity?")
print(formatted_result)